In [1]:
import pandas as pd
import numpy as np

## Üzerinde Çalışacağımız Veriyi Alalım

In [31]:
data = pd.read_excel('example_data.xlsx')
data.duration=pd.to_timedelta(data.duration.astype(str))

In [32]:
data

,date,duration,amplitude
0,2018-09-06 00:00:17,00:00:13,-2
1,2018-09-06 00:00:39,00:00:22,3
2,2018-09-06 00:00:48,00:00:09,-1
3,2018-09-06 00:00:49,00:00:01,4
4,2018-09-06 00:00:50,00:00:01,-7
5,2018-09-06 00:00:51,00:00:01,7
6,2018-09-06 00:00:53,00:00:02,-9
7,2018-09-06 00:01:01,00:00:08,7
8,2018-09-06 00:01:04,00:00:03,-11
9,2018-09-06 00:01:05,00:00:01,5


## Ne Yapmak İstiyoruz?
1. Yukarıdaki veriyi belirli bir koşula göre işaretlemek istiyoruz. Peki neden?
 * **Cevap:** İşaretleri kullanarak bu veri üzerinde çeşitli işlemler gerçekleştirebiliriz
 * **Örnek:** ```python df.groupby('sign').sum()```

## Peki neye göre ve nasıl işaretleyeceğiz?
1. İşaretleme işlemi belirli bir amaca hizmet etmeli
2. Bazı **koşullara** göre yapılmalı.
 * **Örnek:** Amplitude değeri mutlak değerce 4'ten küçük ise, bu dalgayı kendinden önceki dalga ile birleştir.
3. Bu durumda 'hangi veri 4'ten küçük hangisi değil' bunun bir **boolean listesine** ihtiyacımız olacak

## Olası Durumlar Neler?
1. İşaretleme işlemi bir **seriye** veya **dataframe**'e uygulanabilir.
2. Yalnızca '**duration**' ya da yalnızca '**amplitude**' dikkate alınabilir
3. **Her iki sütun** da dikkate alınarak bu işlem gerçekleştirilebilir.

## Koşulu sağlayan boolean listesini bulmak ile İşaretlemek tamamen FARKLI !

### Bir örnek :
* **pozitif amplitude** 4 ten küçün **negatif amplitude 5** ten küçük ise bir önceki dalga ile birleştirelim

In [33]:
amplitude = data.amplitude
amplitude

0     -2
1      3
2     -1
3      4
4     -7
5      7
6     -9
7      7
8    -11
9      5
10    -2
11     8
12    -1
13     2
14    -9
15     6
16   -16
17     4
18    -2
19    19
Name: amplitude, dtype: int64

### Koşulu sağlayan durumların listesini alalım

In [34]:
bool_list= get_bools(amplitude,pozitive_measure=4,negative_measure=5)
bool_list

0     False
1      True
2      True
3      True
4     False
5     False
6     False
7     False
8     False
9     False
10     True
11    False
12     True
13     True
14    False
15    False
16    False
17     True
18     True
19    False
Name: amplitude, dtype: bool

### Şimdi işaretle metoduna amplitude verisini ve bool_list'i verip, koşulu sağlayan işaretleri alacağım

In [35]:
sign_list = sign_data(amplitude,bool_list)
sign_list

0      1
1      1
2      1
3      1
4      1
5      2
6      3
7      4
8      5
9      6
10     6
11     6
12     6
13     6
14     7
15     8
16     9
17     9
18     9
19    10
Name: amplitude, dtype: int32

### SONUÇ

In [37]:
result=data

In [38]:
result['sign']= sign_list
result

,date,duration,amplitude,sign
0,2018-09-06 00:00:17,00:00:13,-2,1
1,2018-09-06 00:00:39,00:00:22,3,1
2,2018-09-06 00:00:48,00:00:09,-1,1
3,2018-09-06 00:00:49,00:00:01,4,1
4,2018-09-06 00:00:50,00:00:01,-7,1
5,2018-09-06 00:00:51,00:00:01,7,2
6,2018-09-06 00:00:53,00:00:02,-9,3
7,2018-09-06 00:01:01,00:00:08,7,4
8,2018-09-06 00:01:04,00:00:03,-11,5
9,2018-09-06 00:01:05,00:00:01,5,6


## Boolean Listesi Alma Kuralları
Yukarıda koşul olarak negatif ve pozitif değerleri manuel girdik. Fakat bu değerler Belirli fonksiyonlara göre alınmalı:
1. negatif_medyan
2. pozitif_medyan
3. genel_medyan ve genel_percentile
4. negatif percentile pozitif percentile
5. ve ileride ekleneilecek yeni fonksiyonlar

#### Aynı Zamanda

1. Yalnızca **amplitude** ya da **duration** a bu koşulları uygulamak isteyebiliriz
2. Her ikisine uygulamak isteyebiliriz
3. Farklı sütunlara farklı koşullar koyabiliriz

## Verileri işaretledikten sonra...

### 1. Her bir işareti kendi arasında toplayıp yeni dalgaları bulabiliriz

In [42]:
from src.change import last_time

In [46]:
result.groupby('sign').agg({'date':last_time,'duration':'sum','amplitude':'sum'})

,date,duration,amplitude
sign,,,
1,2018-09-06 00:00:50,00:00:46,-3
2,2018-09-06 00:00:51,00:00:01,7
3,2018-09-06 00:00:53,00:00:02,-9
4,2018-09-06 00:01:01,00:00:08,7
5,2018-09-06 00:01:04,00:00:03,-11
6,2018-09-06 00:01:24,00:00:20,12
7,2018-09-06 00:02:11,00:00:47,-9
8,2018-09-06 00:02:31,00:00:20,6
9,2018-09-06 00:04:07,00:01:36,-14


### 2. Ortalamasını alabiliriz

In [47]:
result.groupby('sign').mean()

,amplitude
sign,
1,-0.600000
2,7.000000
3,-9.000000
4,7.000000
5,-11.000000
6,2.400000
7,-9.000000
8,6.000000
9,-4.666667


# Koşul Sınıfı

Boolean listesini alabilmek için öyle bir sınıf olmalı ki,
1. Hem pd.**Series** ile hem pd.**DataFrame** ile çalışabilmeli
2. Koşul tipini (**fonksiyonu**) seçebilmeliyiz
3. **+** ve **-** değerler ile ayrı ayrı çalışması gerektiğini ya da gerekmediğini belirtebilmeliyiz
4. Eğer dataframe ile çalışıyorsak **hangi sütunlara** hangi **fonksiyonları** uygulayacağını belirtebilmeliyiz.
5. Bazı fonksiyonlar **parametre** alıyor percentile(0.5) gibi

## İşaretleme Sınıfı

1. İşaretleme amacı farklı olabilir
2. Örnekte yalnızca yeni dalgaları bulduk. Fakat **change** verisinden **amplitude** ve **duratin** hesaplanırken de işaretle tekniği kullanıldı


```python
ornek= KosulSınıfı(data, ['amplitude':median,'duration':percentile],percentile_value=5)
ornek_bool=ornek.get_bool()```

```python
ornek2= KosulSınıfı(data, median)
ornek_bool2=ornek2.get_bool()```


In [ ]:
İsaretleSınıfı(ornek_bool,data)

In [11]:
def get_bools(data : pd.Series, negative_measure : float, pozitive_measure : float) -> pd.Series:
    bools=data.apply(condition_amp,pozitive_measure=pozitive_measure,negative_measure=negative_measure)
    bools[0]=False
    return bools

In [12]:
def condition_amp(x,negative_measure,pozitive_measure):
    if x <0:
        return abs(x)<=negative_measure
    if x >0:
        return abs(x)<=pozitive_measure

In [16]:
def sign_data(data : pd.Series, bools : pd.Series) -> pd.Series:
    a=data.mask(bools).fillna(method='ffill')
    return np.sign(a).diff().ne(0).cumsum()